# Create an Expectation Suite without a sample batch of data

#### This notebook demonstrates how to create an Expectation Suote without a sample of batch of data.
#### This is useful when you do not have sample data or when you know exactly what should be expected of your data and you want the suite authoring process to go faster by skipping the validation step.



In [ ]:
import json
import great_expectations as ge
import great_expectations.jupyter_ux
from great_expectations.core.expectation_configuration import ExpectationConfiguration
from great_expectations.data_context.types.resource_identifiers import ExpectationSuiteIdentifier
from great_expectations.exceptions import DataContextError


## 1. Get a DataContext


#### The following line instantiates a DataContext object from the great_expectations.yml.

#### If you do not have a config file (e.g., you are using Great Expectations in Databricks, AWS EMR Spark), you should replace the following cell with the code that builds configuration and instantiated a DataContext, as explained in this how-to guide: https://docs.greatexpectations.io/en/latest/guides/how_to_guides/configuring_data_contexts/how_to_instantiate_a_data_context_without_a_yml_file.html


In [ ]:
context = ge.data_context.DataContext()

## 2. Get an existing Expectation Suite (by name) or create a new one

In [ ]:
expectation_suite_name = "my_new_expectation_suite" # TODO: replace with a name for your new suite

try:
    suite = context.get_expectation_suite(expectation_suite_name)
except DataContextError:
    suite = context.create_expectation_suite(expectation_suite_name)


## 3. Add Expectations to the suite

#### You are adding Expectations' configuration to the suite. Since there is no sample batch of data, no validation happens during this process. Consult the Expectation Gallery for the types of Expectations that can be added and their arguments: https://greatexpectations.io/expectations


In [ ]:
# create an Expectation

expectation_config = ExpectationConfiguration(
    expectation_type="expect_table_columns_to_match_ordered_list", # this is the name of the expectation type you are adding
    # These are the arguments of the expectation
    # The keys allowed in the dictionary are Parameters and Keyword Arguments of this Expectation Type
    kwargs={
        "column_list": ["account_id", "user_id", "transaction_id", "transaction_type", "transaction_amt_usd"]
    }, 
    # This is how you can optionally add a comment about this expectation. 
    # It will be rendered in Data Docs (see this guide for details: https://docs.greatexpectations.io/en/latest/guides/how_to_guides/configuring_data_docs/how_to_add_comments_to_expectations_and_display_them_in_data_docs.html)
    meta={ 
        "notes": {
            "format": "markdown",
            "content": "Some clever comment about this expectation. **Markdown** `Supported`"
        }
    }
)

# add the Expectation to the suite

suite.add_expectation(expectation_config)


In [ ]:
# just another example

expectation_config = ExpectationConfiguration(
    expectation_type="expect_column_values_to_be_in_set", 
    kwargs={
        "column": "transaction_type",
        "value_set": ["purchase", "refund", "upgrade"]
    }, 
    meta={
        "notes": {
            "format": "markdown",
            "content": "Some clever comment about this expectation. **Markdown** `Supported`"
        }
    }
)

suite.add_expectation(expectation_config)


## 4. Save the Expectation Suite

In [ ]:
context.save_expectation_suite(suite, expectation_suite_name)

## 5. Review the Expectation Suite in Data Docs (optional)

#### Add the HTML view of the new Expectation Suite to Data Docs. It makes it easier to review the suite. 

#### Note: building a Data Docs view of the suite is also a good test. When you add Expectations's configurations to a suite the way shown in this notebook, there is no validation of the arguments that you are providing. Building Data Docs is an opportunity to catch if you forgot a required argument or provided a bad value.


In [ ]:
suite_identifier = ExpectationSuiteIdentifier(expectation_suite_name=expectation_suite_name)
context.build_data_docs(resource_identifiers=[suite_identifier])
context.open_data_docs(suite_identifier)